## CASH for anomaly detection using random search

In [1]:
# Imports
import os, sys, time, warnings, random
from sklearn.model_selection import train_test_split, cross_val_score, \
    StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score, average_precision_score
# Hide warnings
warnings.filterwarnings('ignore')
# Add parent path to sys
p = os.path.abspath('..')
sys.path.insert(1, p)
# Import from parent
from utils import import_dataset, create_search_space

In [2]:
# Import dataset
dataset = '../../data/Annthyroid_withoutdupl_norm_02_v01.arff'
N = 5000 # number of max data points
df = import_dataset(dataset)
if(len(df) > N):
    df = df.sample(n=N)
# Extract X, y
X  = df.iloc[:, :-1]
y = df['outlier']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=10)
# Stats
print("Ratio of outliers in training set:", len(y_train[y_train==1])/len(y_train))
print("Ratio of outliers in test set:", len(y_test[y_test==1])/len(y_test))
print("Training size:", len(X_train))
print("Test size:", len(X_test))

Ratio of outliers in training set: 0.02
Ratio of outliers in test set: 0.02
Training size: 3750
Test size: 1250


In [3]:
# Create algorithm entries and their search spaces
models, search_space, evaluated = create_search_space()
print('Number of algorithms:', len(models))

Number of algorithms: 11


In [4]:
# init
best_model = None
best_params = {}
best_score = 0.0
budget = 600. # time budget in seconds

# Random search
while budget > 0:

    # Step 1 - Sample a model uniformally
    [key] = random.sample(list(models), 1)
    model = models[key]
    hp_space = search_space[key]
    # Step 2 - Sample a configuration from its hyperparam space
    params = hp_space.sample_configuration().get_dictionary()
    model.set_params(**params)

    # cv strategy
    cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25)
    # Evaluate
    start = time.time()
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')
    end = time.time()
    elapsed = end - start
    budget -= elapsed
    evaluated[key] += 1

    # Update best model
    if scores.mean() > best_score:
        best_model = key
        best_score = scores.mean()
        best_params = params

# General results
for key, val in evaluated.items():
    print(key + ':\t', val)
print('Best model:', best_model)
print('Hyperparameter configuration:', best_params)
print('ROC AUC score:', best_score)

cblof:	 12
copod:	 16
ecod:	 9
hbos:	 12
ifor:	 9
knn:	 14
lof:	 11
mcd:	 14
ocsvm:	 15
pca:	 16
sos:	 12
Best model: mcd
Hyperparameter configuration: {'assume_centered': True, 'contamination': 0.06951995318634668, 'support_fraction': 0.7340868676428925}
ROC AUC score: 0.9422140770860775


In [5]:
# Apply best model on test set
final_model = models[best_model].set_params(**best_params)
final_model.fit(X_train, y_train) # refit
y_pred = final_model.predict_proba(X_test)
auc_score = roc_auc_score(y_test, y_pred[:,1])
avg_prec_score = average_precision_score(y_test, y_pred[:,1])
print('Test performance:')
print('\tROC AUC score:\t\t', auc_score)
print('\tAverage pecision score:\t', avg_prec_score)

Test performance:
	ROC AUC score:		 0.9318530612244899
	Average pecision score:	 0.37949347418215923
